In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from tqdm.notebook import tqdm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
from itertools import combinations_with_replacement
from collections import Counter
from scipy import signal
import plotly.express as px


plt.style.use('fivethirtyeight')
np.random.seed(777)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [20]:
def plot_returns(returns):
    normalized_returns = returns / returns.iloc[0]

    fig = go.Figure()

    traces = []
    for column in normalized_returns.columns:
        traces.append(
            go.Scatter(
                x=normalized_returns.index,
                y=normalized_returns[column],
                mode="lines",
                name=column,
            )
        )

    layout = go.Layout(
        title="Normalized Returns Over Time",
        xaxis_title="Date",
        yaxis_title="Normalized Returns",
        width=1000,
        height=500,
    )

    # Create the figure
    fig = go.Figure(data=traces, layout=layout)
    fig.show()


def get_returns_and_covariance(stocks, plot=True, **kwargs):
    stockData = yf.download(stocks, **kwargs)["Adj Close"]
    if plot:
        plot_returns(stockData)
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return meanReturns, covMatrix


def portfolio_annualised_performance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns * weights) * 252
    std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights))) * np.sqrt(252)
    return std, returns


def n_combinations(n, r):
    result = math.factorial(n + r - 1) // (math.factorial(r) * math.factorial(n - 1))
    return result


def get_n_weight_combinations(stocks, num_portfolios):
    n_parts = len(stocks)
    while n_combinations(len(stocks), n_parts) < num_portfolios:
        n_parts += 1
    n_parts += 1
    portfolio_weights = []
    for combination in combinations_with_replacement(stocks, n_parts):
        counts = Counter(combination)
        portfolio_weights.append(np.array([counts[s] / n_parts for s in stocks]))
    return portfolio_weights


def get_portfolio_performances(numPortfolios, meanReturns, covMatrix, riskFreeRate=0):
    portfolio_weights = get_n_weight_combinations(meanReturns.index, numPortfolios)
    actual_num_portfolios = len(portfolio_weights)
    results = np.zeros((3, actual_num_portfolios))
    allocations = []
    for i in tqdm(range(actual_num_portfolios)):
        portfolio_std_dev, portfolio_return = portfolio_annualised_performance(
            portfolio_weights[i], meanReturns, covMatrix
        )
        allocations.append(
            {
                asset: weight
                for asset, weight in zip(meanReturns.index, portfolio_weights[i])
            }
        )
        results[0, i] = portfolio_std_dev
        results[1, i] = portfolio_return
        results[2, i] = (portfolio_return - riskFreeRate) / portfolio_std_dev
    return results, allocations


def find_max_sharpe_portfolio(results, verbose=False):
    selected = results["sharpe_ratio"] == results["sharpe_ratio"].max()

    if verbose:
        print("-" * 80)
        print("Annualised Return:", round(results.loc[selected, "return"].iloc[0], 2))
        print(
            "Annualised Volatility:",
            round(results.loc[selected, "volatility"].iloc[0], 2),
        )
        print("\n")
        allocations = {
            k: round(100 * v, 2)
            for k, v in results.loc[selected, "allocations"].iloc[0].items()
        }
        display(pd.DataFrame(allocations, index=["Allocation"]))
    results.loc[selected, "max_sharpe"] = True
    return results


def find_min_volatility_portfolio(results, verbose=False):
    selected = results["volatility"] == results["volatility"].min()

    if verbose:
        print("-" * 80)
        print("Minimum Volatility Portfolio Allocation\n")
        print("Annualised Return:", round(results.loc[selected, "return"].iloc[0], 2))
        print(
            "Annualised Volatility:",
            round(results.loc[selected, "volatility"].iloc[0], 2),
        )
        print("\n")
        allocations = {
            k: round(100 * v, 2)
            for k, v in results.loc[selected, "allocations"].iloc[0].items()
        }
        display(pd.DataFrame(allocations, index=["Allocation"]))
    results.loc[selected, "min_volatility"] = True
    return results


def find_efficient_portfolios(results, n_bins):
    min_std_dev = results.loc[results["min_volatility"] == True, "volatility"].iloc[0]
    max_std_dev = results.loc[results["max_sharpe"] == True, "volatility"].iloc[0]
    bins = np.linspace(min_std_dev, max_std_dev, n_bins + 1)
    results["vol_bin"] = pd.cut(results["volatility"], bins)

    results["efficient"] = results.groupby("vol_bin", observed=True)[
        "return"
    ].transform(lambda x: x == x.max())
    return results


def plot_allocations(results):
    df = []
    for idx, data in results.loc[results["efficient"] == True].iterrows():
        for asset, weight in data["allocations"].items():
            df.append(
                {
                    "label": str(data["vol_bin"]),
                    "return": data["return"],
                    "volatility": data["volatility"],
                    "asset": asset,
                    "weight": weight,
                }
            )
    df = pd.DataFrame(df)
    fig = px.bar(df, x="label", y="weight", color="asset", width=1000, height=400)
    fig.show()
    
    
def find_edges(results, n_bins):
    _bins = np.linspace(results["volatility"].min(), results["volatility"].max(), n_bins + 1)
    results["_bin"] = pd.cut(results["volatility"], _bins)

    top = results.groupby("_bin", observed=True)[
        "return"
    ].transform(lambda x: x == x.max())
    bottom = results.groupby("_bin", observed=True)[
        "return"
    ].transform(lambda x: x == x.min())
    
    _bins = np.linspace(results["return"].min(), results["return"].max(), n_bins + 1)
    results["_bin"] = pd.cut(results["volatility"], _bins)
    right = results.groupby("_bin", observed=True)[
        "volatility"
    ].transform(lambda x: x == x.max())
    left = results.groupby("_bin", observed=True)[
        "volatility"
    ].transform(lambda x: x == x.min())
    edges = (top | bottom | right | left)
    results = results.drop(columns=["_bin"])
    return edges


def plot_efficient_frontier(results):
    if len(results) > 10_000:
        sampled_results = pd.concat([results.sample(10_000), results.loc[find_edges(results, 50)]])
    else:
        sampled_results = results
        # sampled_results = results.loc[edges].sample(10_000)
        # sampled_results = results.loc[~results.index.isin(edges)].sample(10_000)

    simulated_portfolios = go.Scatter(
        x=sampled_results["volatility"],
        y=sampled_results["return"],
        mode="markers",
        marker=dict(
            color=sampled_results["sharpe_ratio"],
            colorscale="plasma",
            size=10,
            opacity=0.5,
            colorbar=dict(title="Sharpe Ratio"),
        ),
        name="Simulated Portfolios",
    )
    efficient_frontier = go.Scatter(
        x=results.loc[results["efficient"] == True, "volatility"],
        y=signal.savgol_filter(
            results.loc[results["efficient"] == True, "return"], 30, 5
        ),
        line=dict(color="black", width=3, dash="dash"),
        mode="lines",
        name="Efficient Frontier",
    )

    max_sharpe = go.Scatter(
        x=results.loc[results["max_sharpe"] == True, "volatility"],
        y=results.loc[results["max_sharpe"] == True, "return"],
        mode="markers",
        marker=dict(symbol="star", color="red", size=20),
        name="Maximum Sharpe ratio",
    )

    min_volatility = go.Scatter(
        x=results.loc[results["min_volatility"] == True, "volatility"],
        y=results.loc[results["min_volatility"] == True, "return"],
        mode="markers",
        marker=dict(symbol="star", color="green", size=20),
        name="Minimum volatility",
    )
    layout = go.Layout(
        title="Simulated Portfolio Optimization based on Efficient Frontier",
        xaxis=dict(title="annualised volatility"),
        yaxis=dict(title="annualised returns"),
        width=1000,
        height=700,
    )

    fig = go.Figure(
        data=[simulated_portfolios, efficient_frontier, max_sharpe, min_volatility],
        layout=layout,
    )
    fig.add_hline(
        y=risk_free_rate,
        line_width=3,
        line_dash="dash",
        line_color="grey",
        annotation_text="Risk Free Rate",
    )
    fig.update_layout(legend_orientation="h")

    fig.show()
    return None


def simulate_efficient_frontier(
    mean_returns, cov_matrix, num_portfolios, risk_free_rate, plot_scatter=True
):
    results, allocations = get_portfolio_performances(
        num_portfolios, mean_returns, cov_matrix, risk_free_rate
    )
    results = pd.DataFrame(results.T, columns=["volatility", "return", "sharpe_ratio"])
    results["allocations"] = allocations
    results = results.sort_values(by="volatility")

    results = find_max_sharpe_portfolio(results, verbose=plot_scatter)
    results = find_min_volatility_portfolio(results, verbose=plot_scatter)
    results = find_efficient_portfolios(
        results,
        n_bins=30,
    )

    if plot_scatter:
        plot_efficient_frontier(results)
    plot_allocations(results)
    return results


num_portfolios = 25_000
risk_free_rate = 0.04

# stocks = ["VTI", "VNQ", "VXUS", "BND"]
stocks = ["VTI", "TLT", "GLD", "VNQ", "QQQ"]

mean_returns, cov_matrix = get_returns_and_covariance(
    stocks, plot=True, start=f"2015-01-01", end="2024-09-30"
)
results = simulate_efficient_frontier(
    mean_returns, cov_matrix, num_portfolios, risk_free_rate, plot_scatter=True
)

[*********************100%***********************]  5 of 5 completed


  0%|          | 0/31465 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
Annualised Return: 0.15
Annualised Volatility: 0.14




,GLD,QQQ,TLT,VNQ,VTI
Allocation,40.74,59.26,0.0,0.0,0.0


--------------------------------------------------------------------------------
Minimum Volatility Portfolio Allocation

Annualised Return: 0.07
Annualised Volatility: 0.09




,GLD,QQQ,TLT,VNQ,VTI
Allocation,29.63,0.0,37.04,0.0,33.33
